In [1]:
import numpy as np
import pandas as pd
import math
np.set_printoptions(suppress=True)

In [2]:
lst = [[5,-1,5,2], [-1,4,4,4], [2,4,3,3]]
mtrx_R_tutorial = np.array(lst)
r_avg_tut = 3.6

In [3]:
def calc_deviation_b(arr):
    n = len(arr[arr >= 0])
    if n == 0:
        return 0
    sum_arr = 0
    for i in range(len(arr)):
        if arr[i] >= 0:
            sum_arr += arr[i]
    return sum_arr/n

In [4]:
b_u_tut = []
b_i_tut = []
for i in range(len(mtrx_R_tutorial)):
    b_u_tut.append((calc_deviation_b(mtrx_R_tutorial[i]) - r_avg_tut))

mtrx_R_tutorial_tr = mtrx_R_tutorial.T

for i in range(len(mtrx_R_tutorial_tr)):
    b_i_tut.append((calc_deviation_b(mtrx_R_tutorial_tr[i]) - r_avg_tut))

In [5]:
# create R_hat
mtrx_R_hat_tut = np.zeros((len(mtrx_R_tutorial), len(mtrx_R_tutorial[0])))
for i in range(len(mtrx_R_tutorial)):
     for j in range(len(mtrx_R_tutorial[0])):
        # here don't use -1 yet.
        mtrx_R_hat_tut[i,j] = r_avg_tut + b_u_tut[i] + b_i_tut[j]

In [6]:
# create R_tilda
mtrx_R_tilda_tut = mtrx_R_tutorial - mtrx_R_hat_tut

# put -1 after the sub.
# here done manualy but we can extract the indexes which need to be filled and do it for all of them.

mtrx_R_tilda_tut[0,1] = -1
mtrx_R_tilda_tut[1, 0] = -1

In [7]:
def special_inner_profuct(mtrx):
    mtrx_transpose = mtrx.T
    mtrx_R_transpose_copy = mtrx_transpose.copy()
    mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -1, 0, mtrx_R_transpose_copy)
    D = np.zeros((len(mtrx[0]), len(mtrx[0])))
    
    for i in range(len(mtrx_R_transpose_copy)):
        for j in range(len(mtrx_R_transpose_copy)):
            mone = np.dot(mtrx_R_transpose_copy[i], mtrx_R_transpose_copy[j])
            x = 0
            y = 0
            for k in range(len(mtrx_R_transpose_copy[0])):
                
                if mtrx_R_transpose_copy[i][k] != 0 and mtrx_R_transpose_copy[j][k] != 0:
                    x += (mtrx_R_transpose_copy[i][k] ** 2)
                    y += (mtrx_R_transpose_copy[j][k] ** 2)     
            norm1 = x ** 0.5
            norm2 = y ** 0.5
            
            mechane = norm1 * norm2
            D[i,j] = mone / mechane

            
    return D

In [8]:
def get_nearest(similarities, n):
    # n is the number of neighboors we want.
    
    sim_arr = []
    
    for i in range(len(similarities)):
        array = similarities[i].copy()
        array_non_neg = array.copy()
        for j in range(len(array)):
            if int(array_non_neg[j]) == 1 or int(array_non_neg[j]) == -1:
                array_non_neg[j] = 0
            if array_non_neg[j] < 0:
                array_non_neg[j] = array_non_neg[j] * (-1)

        ind = np.argpartition(array_non_neg, -2)[-2:]
        
        top2 = array[sorted(ind)]
        sim_arr.append((top2, sorted(ind)))

    
    return sim_arr

In [11]:
def get_final_R(mtrx_R_hat, mtrx_R_tilda, similiraties):
    nearest = get_nearest(similarities, 2) # get the 2 nearest movies and their indexes.
   
    mtrx_R_final = np.zeros((len(mtrx_R_hat), len(mtrx_R_hat[0])))

    for i in range(len(mtrx_R_hat)):
        for j in range(len(mtrx_R_hat[0])):
#             if (i,j) != (0,1) and (i,j) != (1,0): # this will be replaced by the indexes we extract (cells which need to be filled)
#                 mtrx_R_final[i,j] = mtrx_R_hat[i,j] # only predict for the missing ones.
#                 continue
            
            most_similar_mvs_to_j = nearest[j][1] # similarity indexes.
            r_tilda  = (mtrx_R_tilda[i][most_similar_mvs_to_j])
            mone1 = np.dot(nearest[j][0], r_tilda)
            
            mechane1 = 0
            for k in range(len(nearest[j][0])):
                mechane1 += abs(nearest[j][0][k])
                
            mtrx_R_final[i,j] = mtrx_R_hat[i,j] + mone1/mechane1
    return mtrx_R_final

In [12]:
similarities = special_inner_profuct(mtrx_R_tilda_tut)
m = get_final_R(mtrx_R_hat_tut, mtrx_R_tilda_tut, similarities)
m

array([[4.89296441, 3.4       , 5.64867535, 2.55218982],
       [3.4017589 , 4.9       , 3.59823379, 4.09737221],
       [2.4017589 , 3.9       , 2.64867535, 3.04781018]])